# Imports 📚

In [133]:
import pandas as pd
import numpy as np
import requests
from yahoo_fin.stock_info import get_data
import yfinance as yf
import itertools
from dotenv import load_dotenv, find_dotenv
import os
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import datetime as dt
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email.mime.application import MIMEApplication
from io import BytesIO, StringIO
import tempfile
from os.path import basename

import smtplib
import ssl

# Data 💾

In [2]:
df = pd.read_csv('../raw_data/stocks.csv',
                delimiter=';')

In [3]:
df.head()

,Verwaltung,WKN,Name,Land
0,KSK,EWG2LD,Euwax-Gold,DE
1,KSK,EWG2LD,Euwax-Gold,DE
2,KSK,EWG2LD,Euwax-Gold,DE
3,KSK,A118V8,Pfis ETF-P.LD EO C.B. EOI,IE
4,KSK,A118V8,Pfis ETF-P.LD EO C.B. EOI,IE


# Get data from mail

In [120]:
load_dotenv(find_dotenv())
port = 465  # For SSL
GMAIL = os.environ.get("GMAIL")
homename = os.environ.get("HOMENAME", 'Sender not found')
sender_email = os.environ.get("SENDER")
receiver_email = (os.environ.get("RECEIVER")).split(',') # need list
debug = os.environ.get("DEBUG").lower() in ['true', 'yes', '1', 'most certainly', 'gladly', 'I can hardly disagree']
signature = f"<p>Sincerely, <br>Your Stockist</p>"


In [134]:
def send_email(homename, sender_email, receiver_email, password, port, signature, df, debug):
    """
    Compiles an Email with the email.mime library and sends it through a Google Mail smtp server.
    Takes in variables for the mail content [homename (str), lvl_results (dict), signature (str)]
    and settings to send the email [sender_email, receiver_email, password, port]
    There is a debug argument to test the email function despite a trigger isn't reached.
    """
    print('send email was called.')
    # check, if there is alert level 2 was reached in the checkpoints
    if debug:#or an email was received:
        print('data was requested via mail')
        # init msg
        init = "<p>***Stock prices request***, \
                <br><br>Please find requested historical stock prices attached.<br><br></p>"
        
        # Record the MIME types of text/html.
        text = MIMEMultipart('alternative')
        text.attach(MIMEText(init + signature, 'html', _charset="utf-8"))

        # compile email msg
        now = dt.datetime.now().strftime("%y-%m-%d %H:%M")
        msg = MIMEMultipart('mixed')
        
        # avoid automized email ruling by subject when testing
        if debug:
            msg['Subject'] = f"-TESTRUN- Stock prices {now}"
        else:
            msg['Subject'] = f"-RESULTS- Stock Prices - {now}"
        msg['From'] = f'{homename} <{sender_email}>'
        msg['To'] = ','.join(receiver_email)

        # add all parts to msg
        msg.attach(text)

        # create tables as .csv from temp files
        tempdir = tempfile.gettempdir()
        filename = f'{tempdir}/results.csv'
        
        # filter and save csv tables
        save_file = df.to_csv(filename)
        
        with open(filename, "rb") as fil:
            part = MIMEApplication(
                    fil.read(),
                    Name=basename(filename))

            part['Content-Disposition'] = f'attachment; filename={basename(filename)}'
            msg.attach(part)

        # Create a secure SSL context
        context = ssl.create_default_context()

        with smtplib.SMTP_SSL("smtp.gmail.com", port, context=context) as server:
            server.login(sender_email, password)
            server.sendmail(sender_email, receiver_email, msg.as_string())

        print(f'successfully sent email to {receiver_email}')
    else:
        print(f'There was no request via email.')
        print('Exit function without sending mail.')

In [135]:
send_email(homename=homename, 
           sender_email=sender_email, 
           receiver_email=receiver_email, 
           password=GMAIL, 
           port=port, 
           signature=signature, 
           df=df_done, 
           debug=debug)

send email was called.
data was requested via mail
successfully sent email to ['moritzgeiger@hotmail.de']


# Translate WKN to ticker ⤵️

In [4]:
### ENV VARIABLES
load_dotenv(find_dotenv())
figi_key = os.environ.get("FIGI")

In [5]:
# openfigi
def translate_wkn(df, figi_key=None):
    '''
    translates the wkn to the intl stock ticker code.
    uses openfigi api to get the info
    takes in a df with a "WKN" and "Land" column and figi api key.
    removes duplicates from df.
    if no key provided only 10 requests (rows) can be processed. Else 100 can be processed at a time.
    _____________
    returns enriched df by "ticker" code.
    '''
    # BASE
    openfigi_url = 'https://api.openfigi.com/v1/mapping'
    openfigi_headers = {'Content-Type': 'text/json'}
    

    
    # DF handle (drop duplicates to avoid double requests)
    df_new = df.copy().drop_duplicates(subset=['WKN', 'Land']).dropna(subset=['WKN'])
    
    # INIT JOBS
    jobs = []
    for i, row in df_new.iterrows():
        jobs.append({"idType":"ID_WERTPAPIER","idValue":str(row.WKN)}) # don't define country
    
    # slice jobs into junks of 100 or 10 depending on API key
    if figi_key:
        openfigi_headers['X-OPENFIGI-APIKEY'] = figi_key
        sjobs = [jobs[i:i + 100] for i in range(0, len(jobs), 100)] # very cool slicer!
        
    else:
        sjobs = [jobs[i:i + 10] for i in range(0, len(jobs), 10)]
        
    
    # REQUEST
    responses = []
    
    for jobs in sjobs:
        r = requests.post(url=openfigi_url, headers=openfigi_headers,
                    json=jobs).json()
        
        for i in range(len(r)):
            try:
                responses.append(r[i].get('data')[0].get('ticker'))
            except:
                responses.append('n/a') # fill the list to match the df
    
    df_new['ticker'] = responses
    
    return df_new.reset_index(drop=True)

In [6]:
re = translate_wkn(df, figi_key=figi_key)

In [7]:
re

,Verwaltung,WKN,Name,Land,ticker
0,KSK,EWG2LD,Euwax-Gold,DE,EWG2
1,KSK,A118V8,Pfis ETF-P.LD EO C.B. EOI,IE,LDCE
2,KSK,ETFL35,DeKa Iboxx EO L.GE.COV.DIV.,DE,ETFIBOXG
3,KSK,A14WU5,L+G-L+G Cyb.Sec.U.ETF DZ,IE,ISPY
4,KSK,870747,Microsoft Corp.,US,MSFT
...,...,...,...,...,...
73,KSK,A14R7U,PAYPAL HOLDINGS INC.,US,PYPL
74,KSK,A0RGER,ISHSIII-MSCI EM SM.CAP U.ETF,IE,IEMS
75,KSK,A1J782,ISHS VI-E.MSCI EM MIN.VOL.U.E.,IE,EMMV
76,KSK,555200,Deutsche Post AG,DE,DPW


In [8]:
tickers = [x.split()[0] for x in re.ticker] # some tickers contain spaces

stickers = [' '.join(tickers[i:i + 10]) for i in range(0, len(tickers), 10)]
stickers

['EWG2 LDCE ETFIBOXG ISPY MSFT APX WKL WM SY1 ASML',
 'JPALCAU BLEJPBM XCO2 ALV AAPL ADBE LIN PHBN ADS GRMN',
 'ORLY 7309 NO DG SU ABT IUS5 INRG LXS STT',
 'MMK DCX MC SIE SAP FME MUV2 BAS DG SAN',
 'IFX V PG CSCO MSFT PEP NESR AWC ROG GOOGL',
 'DGE MDT CNR DHR MCD CMCSA OTIS COST ATCOA HCMLF',
 'JPM PEP V CSCO ORSTED NVDA PG NKE EVK AMDGBID',
 'BERTEL LXSGR ESL PYPL IEMS EMMV DPW TRMB']

In [52]:
def history_data(df):
    '''
    searches for historical stock values for designated stocks with the help of yfinance (yahoo).
    takes in a df which contains the column 'ticker'
    ___________
    returns a df with the columns ticker, wkn and the historical values.
    '''
    
    # HANDLE DF
    df_new = df.copy().dropna()
    tickers = [x.split()[0] for x in df_new.ticker] # some tickers contain spaces
    stickers = ' '.join(tickers) # yf requires space separated string
    
    # GET DATA
    df_concat = yf.download(tickers=stickers,
                          period='1y',
                          interval='1d',
                          show_errors=True,
                           progress=True)
    
    # only get the closing price and remove double indexing
    new = df_concat.loc[:,('Close', slice(None))]
    new.columns = [x[1] for x in new.columns] # remove duplex columns
    
    # transform and add old cols
    df_trans = new.T.rename_axis('ticker')#.reset_index()
    df_out = df_trans.merge(df_new[['WKN', 'ticker']], left_index=True, right_on='ticker')
    df_out = df_out.set_index(['WKN', 'ticker'])
    
    # set datetime
    df_out.columns = pd.to_datetime(df_out.columns)
    
    return df_out

In [113]:
history_data(re).to_csv('../raw_data/hist_data.csv')

[*********************100%***********************]  72 of 72 completed

32 Failed downloads:
- ATCOA: No data found, symbol may be delisted
- FME: No data found for this date range, symbol may be delisted
- LXS: No data found for this date range, symbol may be delisted
- NO: No data found for this date range, symbol may be delisted
- ISPY: No data found, symbol may be delisted
- BLEJPBM: No data found, symbol may be delisted
- BERTEL: No data found, symbol may be delisted
- XCO2: No data found, symbol may be delisted
- SY1: No data found, symbol may be delisted
- MUV2: No data found, symbol may be delisted
- APX: No data found for this date range, symbol may be delisted
- IUS5: No data found, symbol may be delisted
- 7309: No data found, symbol may be delisted
- SIE: No data found for this date range, symbol may be delisted
- INRG: No data found for this date range, symbol may be delisted
- LXSGR: No data found, symbol may be delisted
- MMK: No data found for this date range, symbol ma

In [118]:
df_done = history_data(re)

[*********************100%***********************]  72 of 72 completed

32 Failed downloads:
- LXS: No data found for this date range, symbol may be delisted
- ATCOA: No data found, symbol may be delisted
- BERTEL: No data found, symbol may be delisted
- FME: No data found for this date range, symbol may be delisted
- NO: No data found for this date range, symbol may be delisted
- XCO2: No data found, symbol may be delisted
- BLEJPBM: No data found, symbol may be delisted
- ISPY: No data found, symbol may be delisted
- SY1: No data found, symbol may be delisted
- MUV2: No data found, symbol may be delisted
- APX: No data found for this date range, symbol may be delisted
- 7309: No data found, symbol may be delisted
- IUS5: No data found, symbol may be delisted
- SIE: No data found for this date range, symbol may be delisted
- INRG: No data found for this date range, symbol may be delisted
- LXSGR: No data found, symbol may be delisted
- MMK: No data found for this date range, symbol ma